<a href="https://colab.research.google.com/github/monimoreno2905/SegundoParcial/blob/main/RedDependencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. LIBRERIAS Y FUNCIONES GENERALES

In [1]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# en este contexto numpy se utiliza para hacer algunas operaciones matemáticas y definir arreglos
import numpy as np
#En este ejercicio básicamente se utiliza para manipular directorios
import os

from tensorflow.keras import layers

# Con esto se garantiza que produzcan los mismos resultados cada vez que se ejecuta el código
np.random.seed(42)
tf.random.set_seed(42)

# Librerias para la generación de gráficas
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model

In [2]:
#Grafica imágenes originales vs reconstruidas del mnist después de aplicar el modelo encoder, decoder
#Los datos que se le ingresa son: el modelo de reconstrucción, las imágenes originales de las cuales siempre se van a graficar cinco
def show_reconstructions(model, images, n_images=5):
    reconstructions = model.predict(images[:n_images]) #aplica el modelo de autoencoer para sacar imagenes reconstruidas
    fig = plt.figure(figsize=(n_images * 1.5, 3)) #Establece los parámetros de la figura como por ejemplo el tamaño
    for image_index in range(n_images): #En cada iteración, dibuja la imagen original y su reconstrucción correspondiente en la figura.
        plt.subplot(2, n_images, 1 + image_index) #grafica imágenes originales
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index) #grafica reconstrucciones
        plot_image(reconstructions[image_index])

#esta función se utiliza internamente en la función show_reconstructions para graficar las imagenes reconstruidas
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

2. CARGA DE DATOS

In [3]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255 #normalización de los datos
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:] #Se toman solo algunos datos, no todos
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


3. FUNCIONES PARA CALCULAR F Y G de acuerdo a las ecuaciones del Kernel reproductor.

Para no tener problemas en las dimensiones para realizar operaciones más adelante en la función de costos, tenemos;

$f=X^{T}Hα  \in R^{1x1}$

$g=Y^{T}Hβ  \in R^{1x1}$

$C{xy}=X^{T}HY ∈ R^{1x1}$

y la función de costos evaluada

$L(f, g, \lambda, \gamma) = -f C_{XY} g + \frac{\lambda}{2} \|f\|^2_F - 1 + \frac{\gamma}{2} \|g\|^2_G - 1
$

In [5]:
"""Clase para calcular f con base a los datos originales"""
class Calculate_f(layers.Layer):
    """Inicialización
       factor_o=0.1: factor de ajuste para el peso o alguna otra operación en la capa,
       activation=None: Define la función de activación de la capa. Si no se especifica una activación, el valor por defecto es None, lo que significa que la capa no aplicará ninguna función de activación si no se especifica.
    """
    def __init__(self, factor_o=0.1,activation=None, **kwargs):
        super(Calculate_f, self).__init__(**kwargs)
        self.factor_o = factor_o
        self.activation = tf.keras.activations.get(activation)
    """Configura los pesos de la capa perzonalizada (alpha)
    El método build se utiliza en capas personalizadas de Keras para definir las variables que dependen de la forma de entrada (input_shape).
    Se llama cuando la capa se conecta por primera vez a los datos, ya que hasta ese momento, Keras no conoce las dimensiones exactas de entrada.
    """
    def build(self, input_shape):
        n = input_shape[0] #Para saber cuántos datos hay
        self.w = self.add_weight( #Este es el método que añade un tensor de pesos llamado w a la capa.
            shape=(n, 1), #Define la forma del tensor de pesos (alpha)
            initializer="random_normal",
            trainable=True,regularizer=tf.keras.regularizers.OrthogonalRegularizer(factor=self.factor_o),
            constraint=tf.keras.constraints.max_norm(1.)
        )
        super(Calculate_f, self).build(input_shape) #Este es el llamado al método build de la clase base
    """Este método call define cómo se aplica la capa personalizada Calculate_f a los datos de entrada (inputs) durante la ejecución del modelo"""
    def call(self, inputs):
        # Convertir n a float32
        n = tf.shape(inputs)[0]  # Obtener el tamaño del batch dinámicamente
        n = tf.cast(n, tf.float32)
        H = tf.eye(n, dtype=tf.float32) - (1/n) * tf.ones((n, n), dtype=tf.float32)  # Se define H de acuerdo a la cantidad de datos
        f = tf.linalg.matmul(tf.linalg.matmul(tf.transpose(inputs),H), self.w) #Ecuación de f
        if self.activation is not None:
            f = self.activation(f)
        return f

In [9]:
"""Clase para calcular g
Se aplica el proceso anterior pero esta vez para las reconstrucciones que por cierto se despejan de g
"""
class Calculate_g(layers.Layer):
    """Inicialización
       factor_o=0.1: factor de ajuste para el peso o alguna otra operación en la capa,
       activation=None: Define la función de activación de la capa. Si no se especifica una activación, el valor por defecto es None, lo que significa que la capa no aplicará ninguna función de activación si no se especifica.
    """
    def __init__(self, factor_o=0.1,activation=None, **kwargs):
        super(Calculate_g, self).__init__(**kwargs)

        self.factor_o = factor_o
        self.activation = tf.keras.activations.get(activation)
    """Configura los pesos de la capa perzonalizada (beta)
    El método build se utiliza en capas personalizadas de Keras para definir las variables que dependen de la forma de entrada (input_shape).
    Se llama cuando la capa se conecta por primera vez a los datos, ya que hasta ese momento, Keras no conoce las dimensiones exactas de entrada.
    """
    def build(self, input_shape):
        n = input_shape
        self.z = self.add_weight( #Este es el método que añade un tensor de pesos llamado w a la capa.
            shape=(n, 1), #Define la forma del tensor de pesos (beta)
            initializer="random_normal",
            trainable=True,regularizer=tf.keras.regularizers.OrthogonalRegularizer(factor=self.factor_o),
            constraint=tf.keras.constraints.max_norm(1.)
        )
        super(Calculate_g, self).build(input_shape) #Este es el llamado al método build de la clase base
    """Este método call define cómo se aplica la capa personalizada Calculate_g a los datos de entrada (inputs) durante la ejecución del modelo"""
    def call(self, y):
        n = tf.shape(y)[0]  # Obtener el tamaño del batch dinámicamente
        n = tf.cast(n, tf.float32)
        H = tf.eye(n, dtype=tf.float32) - (1/n) * tf.ones((n, n), dtype=tf.float32)  # Definir H según la cantidad de datos
        g = tf.linalg.matmul(tf.linalg.matmul(tf.transpose(y),H), self.z) #Se aplica la ecuación de g
        if self.activation is not None:
            g = self.activation(g)
        #Aqui estoy intentando aplicar algo similar al PCA autoencoder que vimos una vez, despejar y a partir de g pero entro en dilema porque para calcular g necesito y
        y=tf.linalg.matmul(g,tf.transpose(tf.linalg.matmul(H,self.z)))
        y=tf.transpose(y)
        return g

4. DEFINIR EL MODELO

La verdad no se bien como hacer esto, estoy intentando regresar f y g pero tengo el problema del despeje que hice de y en Calculate_g

In [10]:
class Autoencoder(Model):
    """Este código define el constructor (__init__) para una clase llamada Autoencoder"""
    def __init__(self, encoding_dim,factor_o=0.1):
        super(Autoencoder, self).__init__()
        self.encoding_dim = encoding_dim #encoding_dim: Especifica la dimensión del espacio latente donde se codificarán los datos
        self.factor_o=factor_o
        # Encoder layers
        self.encoder_input_layer = layers.Flatten()

        # Decoder layers will be initialized in build()
        #Aquí se define una capa personalizada que será utilizada para conectar el encoder y el decoder.
        self.encoder_decoder_transpose = Calculate_g(self.encoding_dim, factor_o=self.factor_o,activation='linear')
        self.decoder_output_layer = None

    def build(self, input_shape):
        # Now that we have the input shape, initialize decoder layers
        self.encoder_decoder_transpose.build(input_shape[1]*input_shape[2])
        self.decoder_output_layer = layers.Reshape(input_shape[1:])
        super().build(input_shape)


    def call(self, inputs):
        x = self.encoder_input_layer(inputs)
        x = self.encoder_decoder_transpose(x)
        g = self.decoder_output_layer(x)
        return g

In [12]:
#Función para definir el loss
def loss_custom(f,g,Cxy):
  lambda_param = 0.1
  gamma_param = 0.1
  L = -tf.matmul(tf.matmul(f, Cxy), g) + \
    (lambda_param / 2) * tf.square(f) + \
    (gamma_param / 2) * tf.square(g)

In [13]:
# Instantiate the autoencoder
encoding_dim = 64
input_shape = (None, 28, 28, 1)
factor_o = 0.1
#optimizer=keras.optimizers.SGD(learning_rate=0.001)
#optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.1)
pcautoencoder = Autoencoder(encoding_dim)
pcautoencoder.build(input_shape)
pcautoencoder.compile(optimizer='adam', loss=loss_custom) #Se utiliza la función de costos personalizada
pcautoencoder.summary()

TypeError: Calculate_g.__init__() got multiple values for argument 'factor_o'